# TODO:
- [ ] refactor frame creation
- [ ] save labels and label map in vid folder
- [ ] alert if vid with same name already exists
- [ ]  fix logging

# Setup

## imports

In [3]:
import os
import sys
import time
import json
from shutil import copy
from sklearn.utils import shuffle
import datetime as dt

In [4]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D, MaxPooling2D)
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array

Using TensorFlow backend.


In [5]:
import cv2

In [6]:
from contextlib import redirect_stdout

In [7]:
# setup matplotlib to display plots in the notebook
# %matplotlib inline

# third party imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# setup display options
pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 400
pd.options.display.float_format = '{:,.5g}'.format
np.set_printoptions(precision=5, suppress=False)

# setup seaborn to use matplotlib defaults & styles
sns.set()
sns.set(font_scale=1.2)
sns.set_style("whitegrid", {'axes.grid' : False})

## paths

In [8]:
pwd = os.path.dirname(os.getcwd()) + '/'
pwd

'/mnt/seals/'

In [9]:
path_cache = pwd + 'cache/'
path_models = pwd + 'models/'

In [10]:
# for constructing vids in cache REFACTOR
path_data = pwd + 'data/images/'

In [11]:
# folder where we'll store each vid grouped into folders
path_vids = path_cache + 'vids/'

## setup logging

In [12]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])

logger = logging.getLogger()

# helper functions

In [13]:
def plot_pic(imgpath):
    plt.imshow(plt.imread(imgpath))
    plt.show()

# Convert data back into sequence for video classification

## read list of paths across train/val/test and seal / no seal folders

In [14]:
path_data

'/mnt/seals/data/images/'

In [15]:
paths_jpgs = []
for folder, subs, files in os.walk(path_data):        
    for filename in files:
        if filename[-4:] == '.jpg' or  filename[-4:] == 'jpeg':
            paths_jpgs.append(os.path.abspath(os.path.join(folder, filename)))

In [16]:
# create dataframe from paths
dfp = pd.DataFrame(paths_jpgs)
dfp.columns = ['path']
dfp['label'] = dfp['path'].str.split("/").str.get(-2)
dfp['filename'] = dfp['path'].str.split("/").str.get(-1)
dfp.sort_values("filename", inplace=True)
dfp.reset_index(inplace=True,drop=True)
dfp['vid'] = dfp['filename'].str.split("-").str.get(0) + '-' + dfp['filename'].str.split("-").str.get(1)
dfp['seal'] = pd.get_dummies(dfp['label'])['seal']
dfp.to_csv(path_vids + "df.csv")

In [17]:
vids = list(dfp['vid'].unique())

# Create train/test split

## functions to load precomputed sequence data for list of vids

In [18]:
def get_sequence_data_for_vids(list_of_vid_names, sequence_length, pretrained_model_name, pooling):
    """
    Load precomputed sequence features data of given length together with targets [returns data later to train/eval models: x, y]
    for list of vid names and concatenate into one long array
    
    Args:
        list_of_vid_names: name of vid (should already have frames in folder in `/cache/vids/*vid_name*/frames/`)
        sequence_length: length of sequence to fetch precomputed features for
        pretrained_model_name: name of pretrained model whose features should be loaded (assuming these were already precomputed)
        pooling: pooling method used with pretrained model
    
    Returns:
        sequence_features_array_for_all_vids, sequence_targets_array_for_all_vids
    
    """
    
    # create clips of length NUM_FRAMES
    x = []
    y = []
    
    for v, vid_name in enumerate(list_of_vid_names):
        
        path_sequences_features = path_vids + vid_name + '/sequences/features_sequence_' + str(sequence_length) + '_' + pretrained_model_name + '_' + pooling + 'pooling.npy'
        path_sequences_targets = path_vids + vid_name + '/sequences/targets_sequence_' + str(sequence_length) + '.npy'

        # load precomputed features
        features = np.load(path_sequences_features)
        targets = np.load(path_sequences_targets)
        
        x.extend(features)
        y.extend(targets)

    return np.array(x), np.array(y)

## generate train / test split

> todo: write functions to make this quicker

> todo: cross-validation

> todo: experiments with config files and json results with function to aggregate to dataframe

In [19]:
np.random.seed(1337)

In [20]:
ids = list(range(0,len(vids)))
np.random.shuffle(ids)

In [21]:
str(ids)

'[16, 0, 13, 41, 5, 12, 44, 29, 21, 33, 31, 10, 7, 37, 32, 4, 3, 15, 17, 14, 19, 30, 11, 36, 35, 2, 38, 22, 34, 27, 1, 24, 45, 43, 6, 9, 8, 20, 18, 26, 42, 25, 39, 40, 28, 23]'

In [22]:
vids_train = [vids[c] for c in ids[0:40]]
vids_valid = [vids[c] for c in ids[40:44]]
vids_test = [vids[c] for c in ids[44:]]

In [23]:
vids_valid

['s6-1247', 's32-3110', 's45-6301', 's46-8087']

In [24]:
vids_test

['s35-3664', 's30-516']

# Fit models

In [25]:
def fit_model(model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling):

    ###########################
    ### create folder for model 
    ###########################

    path_model = path_models + str(model_id) + '/'
    if not os.path.exists(path_model):
        os.makedirs(path_model)

    ###########################
    ### create train/test split
    ###########################

    x_train, y_train = get_sequence_data_for_vids(vids_train, sequence_length, pretrained_model_name, pooling)
    x_valid, y_valid = get_sequence_data_for_vids(vids_valid, sequence_length, pretrained_model_name, pooling)
    x_test, y_test = get_sequence_data_for_vids(vids_test, sequence_length, pretrained_model_name, pooling)

    # shuffle test and train batches
    x_train, y_train = shuffle(x_train, y_train)
    x_valid, y_valid = shuffle(x_valid, y_valid)

    # CREATE CLASS BALANCE
#     y_train = pd.DataFrame(y_train)
#     keeps = list(y_train[y_train[0]==1].head(int(len(y_train[y_train[0]==1].index)/2)).index)
#     keeps2 = list(y_train[y_train[0] == 0].index)
#     keeps.extend(keeps2)
#     #
#     y_train = y_train.iloc[keeps]
#     y_train = y_train.values
#     x_train = x_train[keeps,:,:]

    NUM_CLASSES = y_train.shape[1]
    NUM_FEATURES = x_train.shape[2]
    SEQ_LENGTH = x_train.shape[1]
    
#     # reshape sequence length 1 into image features if not a sequence
#     if sequence_length == 1:
#         x_train = np.squeeze(x_train, axis=1)
#         x_valid = np.squeeze(x_valid, axis=1)
#         x_test = np.squeeze(x_test, axis=1)



    ##############################
    ### keep track of model params
    ##############################

    # create dict with model parameters
    results = {}

    results['id'] = str(model_id)

    NUM_EPOCHS = 50
    BATCH_SIZE = 32
    results['fit_batch_size'] = BATCH_SIZE
    
    results['fit_num_classes'] = NUM_CLASSES
    results['model_num_features'] = NUM_FEATURES
    results['model_sequence_length'] = SEQ_LENGTH
    
    results['pretrained_model_name'] = pretrained_model_name
    results['pretrained_model_pooling'] = pooling

    results['shape_y_train'] = str(y_train.shape)
    results['shape_x_train'] = str(x_train.shape)
    results['shape_x_valid'] = str(x_valid.shape)
    results['shape_x_test'] = str(x_test.shape)

    results['model_architecture'] = architecture
    results['model_layer_1_sizefactor'] = layer_1_sizefactor
    results['model_layer_2_sizefactor'] = layer_2_sizefactor
    results['model_layer_3_sizefactor'] = layer_3_sizefactor
    results['model_dropout'] = dropout

    ################
    ### define model
    ################

    if architecture == "LSTM":
        # https://github.com/sagarvegad/Video-Classification-CNN-and-LSTM-/blob/master/train_CNN_RNN.py
        model = Sequential()

        # layer 1 (LSTM layer)
        model.add(LSTM(NUM_FEATURES//layer_1_sizefactor, return_sequences=False, dropout=dropout, input_shape=(SEQ_LENGTH, NUM_FEATURES)))

        # layer 2 (dense)
        if layer_2_sizefactor > 0:
            model.add(Dropout(dropout))
            model.add(Dense(NUM_FEATURES//layer_2_sizefactor, activation='relu'))

        # layer 3 (dense)
        if layer_2_sizefactor > 0 and layer_3_sizefactor > 0:
            model.add(Dropout(dropout))
            model.add(Dense(NUM_FEATURES//layer_3_sizefactor, activation='relu'))

        # final layer
        model.add(Dropout(dropout))
        model.add(Dense(NUM_CLASSES, activation='softmax'))

        # define optimizer and compile model
        opt = Adam()
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    if architecture == 'MLP':
        model = Sequential()
        model.add(Flatten(input_shape=(SEQ_LENGTH, NUM_FEATURES)))
        model.add(Dense(NUM_FEATURES//2, activation='relu'))

        if layer_2_sizefactor > 0:
            model.add(Dense(NUM_FEATURES//layer_2_sizefactor, activation='relu'))
            model.add(Dropout(dropout))

        if layer_2_sizefactor > 0 and layer_3_sizefactor > 0:
            model.add(Dense(NUM_FEATURES//layer_3_sizefactor, activation='relu'))
            model.add(Dropout(dropout))

        # final layer
        model.add(Dropout(dropout))
        model.add(Dense(NUM_CLASSES, activation='softmax'))

        # define optimizer and compile model
        opt = Adam()
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    # save model summary to file
    with open(path_model + 'model_summary.txt', 'w') as f:
        with redirect_stdout(f):
            model.summary()

    # track number of params in model
    results['param_count'] = model.count_params()

    #############
    ### fit model
    #############

    # setup training callbacks
    stopper_patience = 10
    results['fit_stopper_patience'] = stopper_patience
    callback_stopper = EarlyStopping(monitor='val_acc', patience=stopper_patience, verbose=0)
    callback_csvlogger = CSVLogger(path_model + 'training.log')
    callback_checkpointer = ModelCheckpoint(path_model +  'model.h5', monitor='val_acc', 
                                 save_best_only=True, verbose=0)
    
    # start time training
    start = dt.datetime.now()
    results['fit_train_dt_start'] = start.strftime("%Y-%m-%d %H:%M:%S")

    history = model.fit(x_train, y_train, 
              validation_data=(x_valid,y_valid),
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              callbacks=[callback_stopper, callback_checkpointer, callback_csvlogger],
              shuffle=True,
              verbose=0)
    
    # end time training
    end = dt.datetime.now()    
    results['fit_train_dt_end']  = end.strftime("%Y-%m-%d %H:%M:%S")
    results['fit_train_dt_duration']  = str((end - start).total_seconds()).split(".")[0]

    # get number of epochs actually trained (might have early stopped)
    epochs_trained = 0
    epochs_trained = callback_stopper.stopped_epoch - stopper_patience

    results['fit_stopped_early'] = True
    if epochs_trained == 0 and len(history.history) > stopper_patience:
        results['fit_stopped_early'] = False
        epochs_trained = NUM_EPOCHS - 1 

    results['fit_num_epochs'] = epochs_trained
    results['fit_val_acc'] = history.history['val_acc'][epochs_trained]
    results['fit_train_acc'] = history.history['acc'][epochs_trained]
    results['fit_val_loss'] = history.history['val_loss'][epochs_trained]
    results['fit_train_loss'] = history.history['loss'][epochs_trained]
    

    #######################
    ### predict on test set
    #######################
    
    
    # start time inference
    start = dt.datetime.now()
    results['fit_test_dt_start'] = start.strftime("%Y-%m-%d %H:%M:%S")

    # calculate predictions on test set
    predictions = model.predict(x_test)
    
    # end time training
    end = dt.datetime.now()    
    results['fit_test_dt_end']  = end.strftime("%Y-%m-%d %H:%M:%S")
    results['fit_test_dt_duration']  = str((end - start).total_seconds()).split(".")[0]

    # calculate test error 
    pdf = pd.DataFrame(predictions)
    pdf.columns = ['noseal','seal']

    # get filenames for predictions
    filenames = []
    for vid_name in vids_test:
        filenames_vid = list(dfp[dfp['vid'] == vid_name]['path'])
        filenames.extend(filenames_vid[sequence_length-1:])
    pdf['filename'] = filenames
    truth = pd.DataFrame(y_test)
    truth.columns = ['truth_noseal','truth_seal']
    truth = truth[['truth_seal']]
    pdf['prediction'] = pdf['seal'].apply(lambda x: round(x))
    pdf = pd.concat([pdf, truth], axis=1)
    pdf['error'] = (pdf['prediction'] != pdf['truth_seal']).astype(int)

    test_acc = 1 - pdf['error'].mean()

    pdf.to_csv(path_model + 'test_predictions.csv')

    results['fit_test_acc'] = 1 - pdf['error'].mean()
    logger.info("model {} test acc: {}".format(model_id, test_acc))

    ###################################
    ### save experiment results to file
    ###################################
    # log results
    logger.info(json.dumps(results))
    
    with open(path_model + 'params.json', 'w') as f:
        json.dump(results, f)

In [26]:
# pretrained_model_names = ["inception_resnet_v2", "inception_v3", "mobilenetv2_1.00_224", "resnet50", "vgg16", "xception"]
# poolings = ['avg','max']
# sequence_lengths = [1, 3, 5, 10, 15, 20, 40]

# architectures = ['LSTM', "MLP"]
# layer_1_sizefactors = [1,2,4,8]
# layer_2_sizefactors = [0,1,2,4,8]
# layer_3_sizefactors = [0,1,2,4,8]
# dropouts = [0, 0.1, 0.2,0.3,0.4,0.5]

In [27]:
pretrained_model_names = ["inception_resnet_v2"]
poolings = ['avg']
sequence_lengths = [1, 3]

architectures = ["MLP", "LSTM"]
layer_1_sizefactors = [1,2,4,8]
layer_2_sizefactors = [0,2,4,8]
layer_3_sizefactors = [0,2,4,8]
dropouts = [0.2, 0.5]

In [28]:
experiment_count_total = 0
for pretrained_model_name in pretrained_model_names:
    for pooling in poolings:
        for sequence_length in sequence_lengths:
            for architecture in architectures:
                for layer_1_sizefactor in layer_1_sizefactors:
                    for layer_2_sizefactor in layer_2_sizefactors:
                        for layer_3_sizefactor in layer_3_sizefactors:
                            for dropout in dropouts:
                                if sequence_length == 1 and architecture == "LSTM":
                                    pass
                                else:
                                    experiment_count_total+=1
experiment_count_total

384

In [29]:
model_id = 1
experiment_count = 1

for pretrained_model_name in pretrained_model_names:
    for pooling in poolings:
        for sequence_length in sequence_lengths:
            for architecture in architectures:
                for layer_1_sizefactor in layer_1_sizefactors:
                    for layer_2_sizefactor in layer_2_sizefactors:
                        for layer_3_sizefactor in layer_3_sizefactors:
                            for dropout in dropouts:
                                
                                # skip LSTM experiement if not a sequence
                                if sequence_length == 1 and architecture == "LSTM":
                                    continue
                                
                                # log experiment
                                param_names = ["model_id", "architecture", "layer_1_sizefactor", "layer_2_sizefactor", "layer_3_sizefactor", "dropout", "sequence_length", "pretrained_model_name", "pooling"]
                                param_values = [str(x) for x in [model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling]]
                                experiment_description = ""
                                for c, p in enumerate(param_names):
                                    experiment_description += p + ': ' + param_values[c] + ', '

                                # only run experiment if results not already computed
                                if not os.path.exists(path_models + str(model_id) + '/params.json'):
                                    # run experiment
                                    logging.info("begin experiment {}/{} - {}".format(experiment_count, experiment_count_total, experiment_description))
                                    fit_model(model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling)
                                
                                experiment_count += 1
                                model_id+=1

# Adhoc experiments

## adhoc - MLP

In [30]:
pretrained_model_names = ["mobilenetv2_1.00_224", "resnet50", "vgg16", "xception"]
poolings = ['avg']
sequence_lengths = [3,5]

architectures = ["MLP"]
layer_1_sizefactors = [2]
layer_2_sizefactors = [8]
layer_3_sizefactors = [0]
dropouts = [0.2]

In [31]:
# increment model id
model_id = 1 + pd.DataFrame(os.listdir(path_models), dtype='int',columns=['model_id']).sort_values("model_id",ascending=False).head(1).values[0][0]
experiment_count = 1

# get total experiments to be run
experiment_count_total = 0
for pretrained_model_name in pretrained_model_names:
    for pooling in poolings:
        for sequence_length in sequence_lengths:
            for architecture in architectures:
                for layer_1_sizefactor in layer_1_sizefactors:
                    for layer_2_sizefactor in layer_2_sizefactors:
                        for layer_3_sizefactor in layer_3_sizefactors:
                            for dropout in dropouts:
                                if sequence_length == 1 and architecture == "LSTM":
                                    pass
                                else:
                                    experiment_count_total+=1

for pretrained_model_name in pretrained_model_names:
    for pooling in poolings:
        for sequence_length in sequence_lengths:
            for architecture in architectures:
                for layer_1_sizefactor in layer_1_sizefactors:
                    for layer_2_sizefactor in layer_2_sizefactors:
                        for layer_3_sizefactor in layer_3_sizefactors:
                            for dropout in dropouts:
                                
                                # skip LSTM experiement if not a sequence
                                if sequence_length == 1 and architecture == "LSTM":
                                    continue
                                
                                # log experiment
                                param_names = ["model_id", "architecture", "layer_1_sizefactor", "layer_2_sizefactor", "layer_3_sizefactor", "dropout", "sequence_length", "pretrained_model_name", "pooling"]
                                param_values = [str(x) for x in [model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling]]
                                experiment_description = ""
                                for c, p in enumerate(param_names):
                                    experiment_description += p + ': ' + param_values[c] + ', '

                                # only run experiment if results not already computed
                                if not os.path.exists(path_models + str(model_id) + '/params.json'):
                                    # run experiment
                                    logging.info("begin experiment {}/{} - {}".format(experiment_count, experiment_count_total, experiment_description))
                                    fit_model(model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling)
                                
                                experiment_count += 1
                                model_id+=1

2018-12-17 23:04:46,678 [MainThread  ] [INFO ]  begin experiment 1/8 - model_id: 471, architecture: MLP, layer_1_sizefactor: 2, layer_2_sizefactor: 8, layer_3_sizefactor: 0, dropout: 0.2, sequence_length: 3, pretrained_model_name: mobilenetv2_1.00_224, pooling: avg, 


KeyboardInterrupt: 

## adhoc - LSTM

In [32]:
pretrained_model_names = ["inception_resnet_v2", "inception_v3", "mobilenetv2_1.00_224", "resnet50", "vgg16", "xception"]
poolings = ['avg']
sequence_lengths = [5,3,10]

architectures = ["LSTM"]
layer_1_sizefactors = [4]
layer_2_sizefactors = [8]
layer_3_sizefactors = [0]
dropouts = [0.2]

In [33]:
# increment model id
model_id = 1 + pd.DataFrame(os.listdir(path_models), dtype='int',columns=['model_id']).sort_values("model_id",ascending=False).head(1).values[0][0]
experiment_count = 1

# get total experiments to be run
experiment_count_total = 0
for pretrained_model_name in pretrained_model_names:
    for pooling in poolings:
        for sequence_length in sequence_lengths:
            for architecture in architectures:
                for layer_1_sizefactor in layer_1_sizefactors:
                    for layer_2_sizefactor in layer_2_sizefactors:
                        for layer_3_sizefactor in layer_3_sizefactors:
                            for dropout in dropouts:
                                if sequence_length == 1 and architecture == "LSTM":
                                    pass
                                else:
                                    experiment_count_total+=1

for pretrained_model_name in pretrained_model_names:
    for pooling in poolings:
        for sequence_length in sequence_lengths:
            for architecture in architectures:
                for layer_1_sizefactor in layer_1_sizefactors:
                    for layer_2_sizefactor in layer_2_sizefactors:
                        for layer_3_sizefactor in layer_3_sizefactors:
                            for dropout in dropouts:
                                
                                # skip LSTM experiement if not a sequence
                                if sequence_length == 1 and architecture == "LSTM":
                                    continue
                                
                                # log experiment
                                param_names = ["model_id", "architecture", "layer_1_sizefactor", "layer_2_sizefactor", "layer_3_sizefactor", "dropout", "sequence_length", "pretrained_model_name", "pooling"]
                                param_values = [str(x) for x in [model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling]]
                                experiment_description = ""
                                for c, p in enumerate(param_names):
                                    experiment_description += p + ': ' + param_values[c] + ', '

                                # only run experiment if results not already computed
                                if not os.path.exists(path_models + str(model_id) + '/params.json'):
                                    # run experiment
                                    logging.info("begin experiment {}/{} - {}".format(experiment_count, experiment_count_total, experiment_description))
                                    try:
                                        fit_model(model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling)
                                    except:
                                        print("cannot train")
                                
                                experiment_count += 1
                                model_id+=1

2018-12-17 23:05:01,282 [MainThread  ] [INFO ]  begin experiment 1/18 - model_id: 472, architecture: LSTM, layer_1_sizefactor: 4, layer_2_sizefactor: 8, layer_3_sizefactor: 0, dropout: 0.2, sequence_length: 5, pretrained_model_name: inception_resnet_v2, pooling: avg, 
2018-12-17 23:06:50,115 [MainThread  ] [INFO ]  model 472 test acc: 0.8711864406779661
2018-12-17 23:06:50,116 [MainThread  ] [INFO ]  {"model_layer_3_sizefactor": 0, "fit_train_acc": 0.8628948998554615, "model_num_features": 1536, "param_count": 3024962, "model_layer_2_sizefactor": 8, "model_sequence_length": 5, "pretrained_model_name": "inception_resnet_v2", "fit_train_dt_duration": "105", "fit_test_dt_start": "2018-12-17 23:06:49", "fit_num_epochs": 3, "fit_val_loss": 0.279447929767142, "model_layer_1_sizefactor": 4, "fit_train_loss": 0.33169885177258346, "fit_test_dt_duration": "0", "model_architecture": "LSTM", "fit_train_dt_start": "2018-12-17 23:05:04", "shape_y_train": "(9686, 2)", "fit_stopper_patience": 10, "pre

2018-12-17 23:23:12,411 [MainThread  ] [INFO ]  begin experiment 7/18 - model_id: 478, architecture: LSTM, layer_1_sizefactor: 4, layer_2_sizefactor: 8, layer_3_sizefactor: 0, dropout: 0.2, sequence_length: 5, pretrained_model_name: mobilenetv2_1.00_224, pooling: avg, 
2018-12-17 23:27:22,784 [MainThread  ] [INFO ]  model 478 test acc: 0.9322033898305084
2018-12-17 23:27:22,785 [MainThread  ] [INFO ]  {"model_layer_3_sizefactor": 0, "fit_train_acc": 0.9514763576541833, "model_num_features": 1280, "param_count": 2100962, "model_layer_2_sizefactor": 8, "model_sequence_length": 5, "pretrained_model_name": "mobilenetv2_1.00_224", "fit_train_dt_duration": "247", "fit_test_dt_start": "2018-12-17 23:27:22", "fit_num_epochs": 21, "fit_val_loss": 0.25963627483989804, "model_layer_1_sizefactor": 4, "fit_train_loss": 0.1287421809108253, "fit_test_dt_duration": "0", "model_architecture": "LSTM", "fit_train_dt_start": "2018-12-17 23:23:14", "shape_y_train": "(9686, 2)", "fit_stopper_patience": 10, 

2018-12-17 23:47:25,427 [MainThread  ] [INFO ]  begin experiment 13/18 - model_id: 484, architecture: LSTM, layer_1_sizefactor: 4, layer_2_sizefactor: 8, layer_3_sizefactor: 0, dropout: 0.2, sequence_length: 5, pretrained_model_name: vgg16, pooling: avg, 
2018-12-17 23:49:34,160 [MainThread  ] [INFO ]  model 484 test acc: 0.9457627118644067
2018-12-17 23:49:34,161 [MainThread  ] [INFO ]  {"model_layer_3_sizefactor": 0, "fit_train_acc": 0.8976873838160615, "model_num_features": 512, "param_count": 336578, "model_layer_2_sizefactor": 8, "model_sequence_length": 5, "pretrained_model_name": "vgg16", "fit_train_dt_duration": "126", "fit_test_dt_start": "2018-12-17 23:49:33", "fit_num_epochs": 9, "fit_val_loss": 0.20289488141967177, "model_layer_1_sizefactor": 4, "fit_train_loss": 0.25678525095558996, "fit_test_dt_duration": "0", "model_architecture": "LSTM", "fit_train_dt_start": "2018-12-17 23:47:26", "shape_y_train": "(9686, 2)", "fit_stopper_patience": 10, "pretrained_model_pooling": "av

In [34]:
pretrained_model_names = ["inception_resnet_v2", "inception_v3", "mobilenetv2_1.00_224", "resnet50", "vgg16", "xception"]
poolings = ['avg']
sequence_lengths = [15, 20, 40]

architectures = ["LSTM"]
layer_1_sizefactors = [4]
layer_2_sizefactors = [8]
layer_3_sizefactors = [0]
dropouts = [0.2]

In [35]:
# increment model id
model_id = 1 + pd.DataFrame(os.listdir(path_models), dtype='int',columns=['model_id']).sort_values("model_id",ascending=False).head(1).values[0][0]
experiment_count = 1

# get total experiments to be run
experiment_count_total = 0
for pretrained_model_name in pretrained_model_names:
    for pooling in poolings:
        for sequence_length in sequence_lengths:
            for architecture in architectures:
                for layer_1_sizefactor in layer_1_sizefactors:
                    for layer_2_sizefactor in layer_2_sizefactors:
                        for layer_3_sizefactor in layer_3_sizefactors:
                            for dropout in dropouts:
                                if sequence_length == 1 and architecture == "LSTM":
                                    pass
                                else:
                                    experiment_count_total+=1

for pretrained_model_name in pretrained_model_names:
    for pooling in poolings:
        for sequence_length in sequence_lengths:
            for architecture in architectures:
                for layer_1_sizefactor in layer_1_sizefactors:
                    for layer_2_sizefactor in layer_2_sizefactors:
                        for layer_3_sizefactor in layer_3_sizefactors:
                            for dropout in dropouts:
                                
                                # skip LSTM experiement if not a sequence
                                if sequence_length == 1 and architecture == "LSTM":
                                    continue
                                
                                # log experiment
                                param_names = ["model_id", "architecture", "layer_1_sizefactor", "layer_2_sizefactor", "layer_3_sizefactor", "dropout", "sequence_length", "pretrained_model_name", "pooling"]
                                param_values = [str(x) for x in [model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling]]
                                experiment_description = ""
                                for c, p in enumerate(param_names):
                                    experiment_description += p + ': ' + param_values[c] + ', '

                                # only run experiment if results not already computed
                                if not os.path.exists(path_models + str(model_id) + '/params.json'):
                                    # run experiment
                                    logging.info("begin experiment {}/{} - {}".format(experiment_count, experiment_count_total, experiment_description))
                                    try:
                                        fit_model(model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling)
                                    except:
                                        print("cannot train")
                                
                                experiment_count += 1
                                model_id+=1

2018-12-18 00:02:14,628 [MainThread  ] [INFO ]  begin experiment 1/18 - model_id: 490, architecture: LSTM, layer_1_sizefactor: 4, layer_2_sizefactor: 8, layer_3_sizefactor: 0, dropout: 0.2, sequence_length: 15, pretrained_model_name: inception_resnet_v2, pooling: avg, 
2018-12-18 00:07:19,615 [MainThread  ] [INFO ]  model 490 test acc: 0.8872727272727272
2018-12-18 00:07:19,616 [MainThread  ] [INFO ]  {"model_layer_3_sizefactor": 0, "fit_train_acc": 0.8829420633211286, "model_num_features": 1536, "param_count": 3024962, "model_layer_2_sizefactor": 8, "model_sequence_length": 15, "pretrained_model_name": "inception_resnet_v2", "fit_train_dt_duration": "297", "fit_test_dt_start": "2018-12-18 00:07:18", "fit_num_epochs": 8, "fit_val_loss": 0.22711319240579805, "model_layer_1_sizefactor": 4, "fit_train_loss": 0.29320081561200456, "fit_test_dt_duration": "1", "model_architecture": "LSTM", "fit_train_dt_start": "2018-12-18 00:02:20", "shape_y_train": "(9286, 2)", "fit_stopper_patience": 10, 

2018-12-18 01:05:12,541 [MainThread  ] [INFO ]  begin experiment 7/18 - model_id: 496, architecture: LSTM, layer_1_sizefactor: 4, layer_2_sizefactor: 8, layer_3_sizefactor: 0, dropout: 0.2, sequence_length: 15, pretrained_model_name: mobilenetv2_1.00_224, pooling: avg, 
2018-12-18 01:12:26,366 [MainThread  ] [INFO ]  model 496 test acc: 0.8909090909090909
2018-12-18 01:12:26,368 [MainThread  ] [INFO ]  {"model_layer_3_sizefactor": 0, "fit_train_acc": 0.9328020676286883, "model_num_features": 1280, "param_count": 2100962, "model_layer_2_sizefactor": 8, "model_sequence_length": 15, "pretrained_model_name": "mobilenetv2_1.00_224", "fit_train_dt_duration": "426", "fit_test_dt_start": "2018-12-18 01:12:24", "fit_num_epochs": 17, "fit_val_loss": 0.19909361744265636, "model_layer_1_sizefactor": 4, "fit_train_loss": 0.17154552143830554, "fit_test_dt_duration": "1", "model_architecture": "LSTM", "fit_train_dt_start": "2018-12-18 01:05:17", "shape_y_train": "(9286, 2)", "fit_stopper_patience": 1

2018-12-18 02:01:47,609 [MainThread  ] [INFO ]  begin experiment 13/18 - model_id: 502, architecture: LSTM, layer_1_sizefactor: 4, layer_2_sizefactor: 8, layer_3_sizefactor: 0, dropout: 0.2, sequence_length: 15, pretrained_model_name: vgg16, pooling: avg, 
2018-12-18 02:12:21,806 [MainThread  ] [INFO ]  model 502 test acc: 0.9418181818181818
2018-12-18 02:12:21,807 [MainThread  ] [INFO ]  {"model_layer_3_sizefactor": 0, "fit_train_acc": 0.9261253499763936, "model_num_features": 512, "param_count": 336578, "model_layer_2_sizefactor": 8, "model_sequence_length": 15, "pretrained_model_name": "vgg16", "fit_train_dt_duration": "629", "fit_test_dt_start": "2018-12-18 02:12:19", "fit_num_epochs": 37, "fit_val_loss": 0.16595764376828262, "model_layer_1_sizefactor": 4, "fit_train_loss": 0.1961651289101025, "fit_test_dt_duration": "1", "model_architecture": "LSTM", "fit_train_dt_start": "2018-12-18 02:01:49", "shape_y_train": "(9286, 2)", "fit_stopper_patience": 10, "pretrained_model_pooling": "

# Examine results

In [36]:
results = []
for folder, subs, files in os.walk(path_models):        
    for filename in files:
        if filename == 'params.json':
            with open(os.path.abspath(os.path.join(folder, filename))) as f:
                data = json.load(f)
            results.append(data)

results = pd.DataFrame(results)

results.sort_values("fit_test_acc", inplace=True, ascending=False)

In [37]:
results.sort_values("fit_val_acc", inplace=True, ascending=False)

In [38]:
results.head(20).T

,284,325,75,345,78,176,22,180,375,54,369,414,243,139,356,278,14,418,302,394
dt_duration_seconds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155,829,734,NaN,NaN,NaN,794,423,NaN,NaN,278,NaN,126
dt_end,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-16 19:15:04,2018-12-16 22:20:45,2018-12-16 23:24:46,NaN,NaN,NaN,2018-12-16 21:03:18,2018-12-17 09:56:39,NaN,NaN,2018-12-17 09:24:11,NaN,2018-12-16 17:37:25
dt_start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-16 19:12:29,2018-12-16 22:06:55,2018-12-16 23:12:32,NaN,NaN,NaN,2018-12-16 20:50:04,2018-12-17 09:49:36,NaN,NaN,2018-12-17 09:19:33,NaN,2018-12-16 17:35:18
fit_batch_size,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32
fit_num_classes,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
fit_num_epochs,28,9,11,37,32,5,22,14,35,28,15,5,22,35,36,16,17,20,18,16
fit_stopped_early,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
fit_stopper_patience,10,10,10,10,10,10,10,5,10,10,10,10,10,10,10,10,10,10,10,5
fit_test_acc,0.92,0.94717,0.97333,0.94182,0.89831,0.92982,0.86441,0.82274,0.92308,0.86288,0.94983,0.94314,0.88302,0.87291,0.88294,0.91228,0.89091,0.87625,0.89298,0.92977
fit_test_dt_duration,2,1,2,1,4,0,0,NaN,NaN,NaN,0,4,2,NaN,NaN,0,1,NaN,2,NaN


In [ ]:
results.to_csv(pwd+'results.csv')

# Analyze results of balanced vs unbalanced fits

In [ ]:
# results2 = pd.read_csv(pwd + 'results/results1.csv', index_col=0)

# results['type'] = 'unbalanced'
# results2['type'] = 'balanced'

# results = pd.concat([results,results2],axis=0)

# results.head().T